In [1]:
import streamlit as st
import duckdb
import requests
import osmnx as ox
import numpy as np
import pandas as pd
from shapely.geometry import Point
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from geopy.geocoders import Nominatim
import altair as alt
from shapely.geometry import Point
import geopandas as gpd
from math import radians, cos, sin, asin, sqrt
import ee
import time, requests
import os
from io import BytesIO
import zipfile
import tempfile

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-shaddie77')

CENSUS_API_KEY = st.secrets.get("CENSUS_API_KEY", "")  # set via Streamlit secrets or replace string
HUGGINGFACE_DATASET = "foursquare/fsq-os-places"
HF_PARQUET_API = f"https://datasets-server.huggingface.co/parquet?dataset={HUGGINGFACE_DATASET}"

In [3]:
# ------------------------
# PARAMETERS
# ------------------------
radius_m = 100       # Neighborhood radius
cr = 50              # Subcircle radius
radius_c = 50        # Candidate facility radius (for city split)
city_name = "New York, NY"


# --- Parameters
location_name = "Times Square, New York, NY"
# location_name = "New York, NY"
# --- Geocode location
lat, lon = ox.geocoder.geocode(city_name)

In [4]:
def generate_city_candidate_locations(location_name, radius_c):
    # Use OSMnx to get city polygon
    print(f'Generating candidate locations ...')
    gdf = ox.geocode_to_gdf(location_name)
    city_poly = gdf.geometry.iloc[0]
    bounds = city_poly.bounds

    step = radius_c * 1.5
    deg_step = step / 111_320
    print(f'size of generate city loop: {len(np.arange(bounds[1], bounds[3], deg_step))}')
    candidates = []
    count = 0
    for lat in np.arange(bounds[1], bounds[3], deg_step):
        for lon in np.arange(bounds[0], bounds[2], deg_step):
            p = Point(lon, lat)
            if city_poly.contains(p):
                candidates.append((lat, lon))
        # print(f'current count of generate city loop: {count}')
        count += 1
    return candidates[0:99]

In [5]:
# Placeholder for median income retrieval
def get_median_income_by_point(lat, lon, radius):
    # TODO: Replace with buffered multi-tract ACS query
    print(f'Getting media_income ...')
    """Use FCC API to find block FIPS then Census ACS to fetch B19013_001E (median household income)."""
    if not CENSUS_API_KEY:
        raise RuntimeError("CENSUS_API_KEY is not set. Put your key in Streamlit secrets or set variable.")
    # FCC to get block FIPS
    j = get_fips_from_coords(lat, lon, retries=3, wait=5)
    block_fips = j.get("Block", {}).get("FIPS")
    if not block_fips:
        return None
    state_fips = block_fips[0:2]
    county_fips = block_fips[2:5]
    tract_fips = block_fips[5:11]

    headers = {
        "X-API-Key": CENSUS_API_KEY
    }

    acs_url = (
        "https://api.census.gov/data/2022/acs/acs5"
        f"?get=B19013_001E&for=tract:{tract_fips}&in=state:{state_fips}%20county:{county_fips}&key={CENSUS_API_KEY}"
    )
    r2 = requests.get(acs_url, headers=headers, timeout=50)
    r2.raise_for_status()
    arr = r2.json()
    if len(arr) < 2:
        return None
    val = arr[1][0]
    try:
        return float(val) if val not in (None, "", "null") else None
    except Exception:
        return None


In [6]:
candidates = generate_city_candidate_locations(city_name, radius_c)
print(f'size of candidates {len(candidates)}')
print(f'element of candidates {candidates[0]}')

Generating candidate locations ...
size of generate city loop: 655
size of candidates 99
element of candidates (40.477251733381244, -74.22717753108134)


In [7]:
def generate_circle_points(center_lat, center_lon, big_radius, N=10):
    """
    Generates subcircle centers within big circle.
    small_radius is chosen so that the number of subcircles <= N.
    """
    print(f'Generating circle points with max {N} subcircles')

    def count_points_for_radius(small_radius):
        step = small_radius * 1.5
        deg_step = step / 111_320
        count = 0
        for lat in np.arange(center_lat - big_radius/111_320,
                             center_lat + big_radius/111_320, deg_step):
            for lon in np.arange(center_lon - big_radius/111_320,
                                 center_lon + big_radius/111_320, deg_step):
                if haversine(center_lon, center_lat, lon, lat) <= big_radius:
                    count += 1
        return count

    # Binary search for the largest small_radius that satisfies count <= N
    low, high = 1.0, big_radius  # meters
    best_radius = low
    for _ in range(30):  # enough iterations for sub-meter precision
        mid = (low + high) / 2
        if count_points_for_radius(mid) <= N:
            best_radius = mid
            low = mid
        else:
            high = mid
    low = 50
    best_radius = 50
    print(f'low, high {low}, {high}')

    # Now generate the actual points with the chosen radius
    step = best_radius * 1.5
    deg_step = step / 111_320
    points = []
    print(f'looper {len(np.arange(center_lat - big_radius/111_320, center_lat + big_radius/111_320, deg_step) )}')
    for lat in np.arange(center_lat - big_radius/111_320,
                         center_lat + big_radius/111_320, deg_step):
        print(f'looper 2 {len(np.arange(center_lon - big_radius/111_320, center_lon + big_radius/111_320, deg_step))}')
        for lon in np.arange(center_lon - big_radius/111_320,
                             center_lon + big_radius/111_320, deg_step):
            # print(f'haversine condition {haversine(center_lat, center_lon, lat, lon)} {big_radius}')
            if haversine(center_lat, center_lon, lat, lon) <= big_radius:
                print(f'haversine condition {haversine(center_lat, center_lon, lat, lon)} {big_radius}')
                points.append((lat, lon))

    print(f"Chosen small_radius: {best_radius:.2f} m, generated {len(points)} subcircles")
    return points[0:99] # , best_radius

In [8]:
def haversine(lon1, lat1, lon2, lat2):
    # Distance in meters
    # print(f'Computing haversine ...')
    R = 6371000
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return R * 2 * asin(sqrt(a))

In [9]:
def build_features_for_location(lat, lon, radius_m, cr):
    print(f'Building features for location ...')
    neighborhood_points = generate_circle_points(lat, lon, radius_m, cr)
    print(f'Number of neighborhood points {len(neighborhood_points)}')
    features = []
    for (lat_i, lon_i) in neighborhood_points:
        pop = get_population_density_gee(lat_i, lon_i, cr)
        osm_poi = get_osm_poi_density(lat_i, lon_i, cr)
        fsq_poi = get_fsq_count(lat_i, lon_i, cr)
        # print(f'pop, fsq_poi {pop} {fsq_poi}')
        income = get_median_income_by_point(lat_i, lon_i, cr)
        # income = get_median_income_with_radius(lat, lon)
        features.append({
            "lat": lat_i,
            "lon": lon_i,
            "population_density": pop,
            "osm_poi_density": osm_poi,
            "fsq_poi_count": fsq_poi,
            "median_income": income
        })
    return pd.DataFrame(features)

In [10]:
# def get_osm_poi_density(lat, lon, radius):
#     print(f'Getting poi density')
#     tags = {"amenity": True}
#     pois = ox.features_from_point((lat, lon), tags=tags, dist=radius)
#     return len(pois)

In [11]:
# import osmnx as ox
# from osmnx._errors import InsufficientResponseError

# def get_osm_poi_density(lat, lon, radius, max_expand=5, expand_factor=2):
#     """
#     Get POI density around (lat, lon). If none are found, expand the radius
#     up to max_expand times until results are found.
    
#     Parameters
#     ----------
#     lat, lon : float
#         Center point coordinates
#     radius : float
#         Search radius in meters
#     max_expand : int
#         Number of times to expand search if no results
#     expand_factor : float
#         Factor by which to expand the radius each retry
    
#     Returns
#     -------
#     int : count of POIs
#     """
#     print(f"Getting POI density at ({lat}, {lon}), radius={radius}m")

#     tags = {"amenity": True}
#     attempt_radius = radius

#     for attempt in range(max_expand + 1):
#         try:
#             pois = ox.features_from_point((lat, lon), tags=tags, dist=attempt_radius)
#             if pois is not None and len(pois) > 0:
#                 return len(pois)
#             else:
#                 print(f"No POIs found at radius {attempt_radius}m, expanding search...")
#         except InsufficientResponseError:
#             print(f"OSM insufficient response at radius {attempt_radius}m, expanding search...")

#         # Expand radius and retry
#         attempt_radius *= expand_factor

#     # If nothing found after all expansions
#     print(f"No POIs found after expanding search up to {attempt_radius}m.")
#     return 0


In [12]:
# def get_population_density_gee(lat, lon, radius_m):

#     print(f'Getting population density gee')
#     dataset = ee.ImageCollection("WorldPop/GP/100m/pop") \
#         .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
#         .first()
#     point = ee.Geometry.Point(lon, lat)
#     region = point.buffer(radius_m).bounds()
#     stats = dataset.reduceRegion(
#         reducer=ee.Reducer.mean(),
#         geometry=region,
#         scale=100,
#         maxPixels=1e9
#     )
#     ans = stats.getInfo().get('population', 0)
#     return 0 if ans == None else 0

In [13]:
def get_fsq_count(lat, lon, r):
    print(f'Getting Four Square Count')
    api_url = "https://datasets-server.huggingface.co/parquet?dataset=foursquare/fsq-os-places"
    j = requests.get(api_url).json()
    parquet_urls = [f['url'] for f in j.get('parquet_files', []) if f['split'] == 'train']
    if not parquet_urls:
        return 0
    url = parquet_urls[0]

    con = duckdb.connect()
    con.execute("INSTALL httpfs;")
    con.execute("LOAD httpfs;")

    deg = r / 111_320
    min_lat, max_lat = lat - deg, lat + deg
    min_lon, max_lon = lon - deg, lon + deg

    query = f"""
    SELECT COUNT(*) as count
    FROM '{url}'
    WHERE latitude BETWEEN {min_lat} AND {max_lat}
      AND longitude BETWEEN {min_lon} AND {max_lon}
    """
    res = con.execute(query).df()
    return int(res['count'][0]) if res.shape[0] else 0


In [14]:
import osmnx as ox
import ee
from geopy.geocoders import Nominatim

# ee.Initialize()

# ----------------------------
# Helper: get nearest town/city center
# ----------------------------
def get_nearest_place_coords(lat, lon):
    """
    Returns (lat, lon) of nearest city/town/village center to the input coordinates.
    Uses Nominatim reverse + forward geocoding.
    """
    geolocator = Nominatim(user_agent="geo-fallback-app")

    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location and "address" in location.raw:
            addr = location.raw["address"]
            place = addr.get("city") or addr.get("town") or addr.get("village")
            if place:
                place_loc = geolocator.geocode(place)
                if place_loc:
                    return (place_loc.latitude, place_loc.longitude)
    except Exception as e:
        print(f"Geocoding fallback failed: {e}")

    return None


# ----------------------------
# POI Density (with fallback)
# ----------------------------
def get_osm_poi_density(lat, lon, radius, max_expand=3, expand_factor=2):
    """
    Get POI density from OSM.
    Expands radius if no POIs found, and falls back to nearest
    town/city center if still empty.
    """
    print(f"Getting POI density at ({lat}, {lon}), radius={radius}m")

    attempt_radius = radius
    tags = {"amenity": True}

    # Try with expanding radius
    for attempt in range(max_expand + 1):
        try:
            pois = ox.features_from_point((lat, lon), tags=tags, dist=attempt_radius)
            if len(pois) > 0:
                print(f'Found osm_pois {len(pois)}')
                return len(pois)
            else:
                print(f"No POIs at radius {attempt_radius}m, expanding search...")
        except Exception as e:
            print(f"OSM query failed at radius {attempt_radius}m: {e}")

        attempt_radius *= expand_factor

    # Fallback to nearest city/town
    print("No POIs found after expansions. Falling back to nearest town/city center...")
    fallback_coords = get_nearest_place_coords(lat, lon)
    if fallback_coords:
        try:
            pois = ox.features_from_point(fallback_coords, tags=tags, dist=radius)
            if len(pois) > 0:
                print(f'Found osm poi {len(pois)}')
                return len(pois)
        except Exception as e:
            print(f"OSM fallback query failed: {e}")

    print("No POIs found, even after fallback.")
    return 0


# ----------------------------
# Population Density (with fallback)
# ----------------------------
def get_population_density_gee(lat, lon, radius_m, max_expand=3, expand_factor=2):
    """
    Get population density from WorldPop using Earth Engine.
    Expands radius if no values are found, and falls back to nearest
    city/town center if still empty.
    """
    print(f"Getting population density at ({lat}, {lon}), radius={radius_m}m")

    dataset = ee.ImageCollection("WorldPop/GP/100m/pop") \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .first()

    attempt_radius = radius_m

    # Try with expanding radius
    for attempt in range(max_expand + 1):
        try:
            point = ee.Geometry.Point(lon, lat)
            region = point.buffer(attempt_radius).bounds()
            stats = dataset.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=100,
                maxPixels=1e9
            )
            result = stats.getInfo()
            pop_val = result.get('population', None)
            if pop_val is not None:
                print(f'Found pop_val {pop_val}')
                return pop_val
            else:
                print(f"No population data at radius {attempt_radius}m, expanding search...")
        except Exception as e:
            print(f"GEE query failed at radius {attempt_radius}m: {e}")

        attempt_radius *= expand_factor

    # Fallback to nearest city/town
    print("No population found after expansions. Falling back to nearest town/city center...")
    fallback_coords = get_nearest_place_coords(lat, lon)
    if fallback_coords:
        try:
            point = ee.Geometry.Point(fallback_coords[::-1])  # (lon, lat)
            region = point.buffer(radius_m).bounds()
            stats = dataset.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=region,
                scale=100,
                maxPixels=1e9
            )
            result = stats.getInfo()
            pop_val = result.get('population', None)
            if pop_val is not None:
                print(f'Found pop_val {pop_val}')
                return pop_val
        except Exception as e:
            print(f"GEE fallback query failed: {e}")

    print("No population data found, even after fallback.")
    return 0

In [15]:
def get_fips_from_coords(lat, lon, retries=3, wait=5):
    url = "https://geo.fcc.gov/api/census/block/find"
    params = {"latitude": lat, "longitude": lon, "format": "json"}
    for i in range(retries):
        try:
            r = requests.get(url, params=params, timeout=20)
            r.raise_for_status()
            return r.json()
        except requests.exceptions.HTTPError as e:
            if r.status_code == 502 and i < retries - 1:
                time.sleep(wait)
                continue
            raise

In [16]:
def revenue_estimation(lat, lon):
    print(f'Revenue estimation ...')
    # print(f'Revenue Est pop density gee {get_population_density_gee(lat, lon, 500)}')
    # print(f'Fsq count {get_fsq_count(lat, lon, 500)}')
    # print(f'osm poi density {get_osm_poi_density(lat, lon, 500)}')
    print(f'Get median income {get_median_income_by_point(lat, lon, 500)}')
    return (get_population_density_gee(lat, lon, 500) * 2 +
            get_osm_poi_density(lat, lon, 500) * 100 +
            get_fsq_count(lat, lon, 500) * 50 
            )

In [17]:
rows = []
cnt_ = 0
for lat, lon in candidates:
    # print(f'cnt_ {cnt_}')
    X_df = build_features_for_location(lat, lon, radius_m, cr)
    # Aggregate neighborhood features (mean as example)
    agg = X_df.mean(numeric_only=True).to_dict()
    Y = revenue_estimation(lat, lon)
    # median_income = get_median_income_by_point(lat, lon, 500) * 0.01
    print(f'revenue {Y}')
    agg["lat"], agg["lon"], agg["revenue"] = lat, lon,  Y, 
    # agg["lat"], agg["lon"] = lat, lon
    rows.append(agg)
    cnt_ += 1

Building features for location ...
Generating circle points with max 50 subcircles
low, high 50, 100
looper 3
looper 2 3
haversine condition 36.889138010898556 100
haversine condition 56.84741204648442 100
looper 2 3
haversine condition 25.878023753085042 100
haversine condition 50.4030067947202 100
looper 2 3
haversine condition 28.423571051562558 100
haversine condition 51.75612163241232 100
Chosen small_radius: 50.00 m, generated 6 subcircles
Number of neighborhood points 6
Getting population density at (40.47635342220625, -74.22740210887508), radius=50m
No population data at radius 50m, expanding search...
No population data at radius 100m, expanding search...
No population data at radius 200m, expanding search...
No population data at radius 400m, expanding search...
No population found after expansions. Falling back to nearest town/city center...
No population data found, even after fallback.
Getting POI density at (40.47635342220625, -74.22740210887508), radius=50m
OSM query fai

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
len(rows)

92

In [19]:
rows[0:2]

[{'lat': 40.477251733381244,
  'lon': -74.22717753108134,
  'population_density': 0.0,
  'osm_poi_density': 2.0,
  'fsq_poi_count': 0.0,
  'median_income': -222147698.0,
  'revenue': 800},
 {'lat': 40.477251733381244,
  'lon': -74.22650379770009,
  'population_density': 0.0,
  'osm_poi_density': 0.0,
  'fsq_poi_count': 0.0,
  'median_income': -333277440.0,
  'revenue': 800}]

In [20]:
df = pd.DataFrame(rows)

print(f'columns of the df {df.columns}')

columns of the df Index(['lat', 'lon', 'population_density', 'osm_poi_density', 'fsq_poi_count',
       'median_income', 'revenue'],
      dtype='object')


In [21]:
# Fit regression
X = df[["population_density", "osm_poi_density", "fsq_poi_count", "median_income"]]
y = df["revenue"]
model = LinearRegression().fit(X, y)

In [22]:
model

LinearRegression()

In [23]:
print("Regression coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Regression coefficients: [0.00000000e+00 3.90089836e+03 0.00000000e+00 6.59451413e-05]
Intercept: 55800.706427168676
